In [1]:
# Does not need to be executed if
# ~/.ipython/profile_default/ipython_config.py
# exists and contains:
# c.InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [2]:
import zipfile
with zipfile.ZipFile('Illustrations.zip') as illustrations:
    illustrations.extractall('.')

In [3]:
import ipywidgets as widgets
from copy import deepcopy
from neotermcolor import colored
from random import shuffle
import neotermcolor
neotermcolor.tty_aware = False

ModuleNotFoundError: No module named 'ipywidgets'

In [4]:
class TraceSorting:
    def __init__(self, sorting_algorithm, nb_of_steps, batchersort=False):
        self.sorting_algorithm = sorting_algorithm
        self.nb_of_steps = nb_of_steps
        self.list = list(range(1, 9)) if batchersort else list(range(1, 11))
        self.tracer = sorting_algorithm(self.list)
        self.play = widgets.Play(interval=500, show_repeat=False)
        self.play.max = self.nb_of_steps(list(self.list))
        self.traces = widgets.Output()
        self.displayed_list = widgets.Text(self.display_list(),
                                           description='List to sort:'
                                          )
        self.default_order = widgets.Dropdown(options=['Sorted','Reversed',
                                                       'Random'
                                                      ]
                                             )
        self.length = widgets.Dropdown(options=[2, 4, 8, 16],
                                       value=8,
                                       description='List size',
                                      ) if batchersort\
                      else widgets.IntSlider(value=10, min=2, max=16,
                                             description='List size'
                                            )
        self.pause = widgets.FloatSlider(value=1, min=0.5, max=5, step=.5,
                                         description='Pause (s.)'
                                        )
        self.tracing_tab = widgets.VBox([self.play, self.traces])
        self.setting_tab = widgets.HBox([widgets.VBox([self.length,
                                                       self.pause
                                                      ]
                                                     ),
                                         widgets.VBox([self.default_order,
                                                       self.displayed_list
                                                      ]
                                                     )
                                        ]
                                       )
        self.tabs = widgets.Tab([self.tracing_tab, self.setting_tab])
        self.tabs.set_title(0, 'Trace algorithm')
        self.tabs.set_title(1, 'Settings')
        self.pause.observe(self.choose_waiting_time, 'value')
        self.length.observe(self.choose_list, 'value')
        self.default_order.observe(self.choose_list, 'value')
        self.play.observe(self.trace, 'value')

    def reinitialise_tracer(self):
        self.play.max = self.nb_of_steps(list(self.list))
        self.tracer.close()
        self.tracer = self.sorting_algorithm(list(self.list))

    def display_list(self):
        return ' '.join(str(e) for e in self.list)

    def trace(self, _):
        with self.traces:
            if not self.play.value:
                print()
            else:
                next(self.tracer)
            if not self.play.value or self.play.value == self.play.max:
                self.reinitialise_tracer()

    def choose_waiting_time(self, _):
        self.play.interval = self.pause.value * 1000

    def choose_list(self, _):
        self.list = list(range(1, self.length.value + 1))
        if self.default_order.value == 'Reversed':
            self.list.reverse()
        elif self.default_order.value == 'Random':
            while True:
                shuffle(self.list)
                if self.length.value == 2\
                or self.list != sorted(range(1, self.length.value + 1))\
                   and self.list !=\
                              list(reversed(range(1, self.length.value + 1))):
                    break
        self.displayed_list.value = self.display_list()
        self.play.value = 0
        self.reinitialise_tracer()

In [5]:
def initialise_colouring(L, blue=True):
    return {i: blue and {'color': 'blue'} or {} for i in range(len(L))}

def display(L, item_colours, last_in_line=True):
    print(' '.join(colored(L[i], **item_colours[i]) for i in range(len(L))
                  ), end=last_in_line and '\n' or '    '
         )

def display_2(L1, item_colours_1, L2, item_colours_2):
    display(L1, item_colours_1, False)
    display(L2, item_colours_2)